In [ ]:
import cv2
import numpy as np
import os
import pickle
import mediapipe as mp

In [ ]:
# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

In [ ]:
# Load the trained model
model_filepath = r"C:\Users\HP\Contacts\Desktop\Gesture sense\Model\hand_gesture_model_mediapipe.pkl"
with open(model_filepath, 'rb') as f:
    model = pickle.load(f)

In [ ]:
# Define function to preprocess test image
def preprocess_image(image):
    # Resize the image to match the input size during training
    target_size = (150, 150)
    resized_image = cv2.resize(image, target_size)
    return resized_image

In [ ]:
# Define function to predict gesture
def predict_gesture(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    # Convert image to RGB (MediaPipe requires RGB format)
    image_rgb = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2RGB)
    # Detect hand landmarks using MediaPipe
    results = hands.process(image_rgb)
    # Check if hand landmarks are detected
    if results.multi_hand_landmarks:
        hand_landmarks = []
        # Extract hand landmarks' coordinates
        for hand_landmark in results.multi_hand_landmarks:
            landmark_list = [(lm.x, lm.y, lm.z) for lm in hand_landmark.landmark]
            hand_landmarks.append(landmark_list)
        # Flatten the hand landmarks
        hand_landmarks_flat = np.array(hand_landmarks).reshape(len(hand_landmarks), -1)
        # Predict gesture using the trained model
        gesture_prediction = model.predict(hand_landmarks_flat)
        return gesture_prediction
    else:
        return None

In [ ]:
# Initialize camera capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

In [ ]:
# Perform gesture prediction
gesture_prediction = predict_gesture(frame)

    if gesture_prediction is not None:
        cv2.putText(frame, f"Gesture: {gesture_prediction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

In [ ]:
  # Check for 'q' key to exit
if cv2.waitKey(1) & 0xFF == ord('q'):
    break

# Release the capture
cap.release()
cv2.destroyAllWindows()
